## Assignmnet No.2
### NLP - Sentiment Analysis

Submitted by : **Saikiran N. Pasikanti**<br>
compiled online at colab.research.google.com with GPU kernel

# Sentiment Analysis on NLTK - Twitter Samples

**Task2** :The goal here is to predict the sentiments behind tweets

**Twitter :** People tweet about their life, events and express opinion about various topics using text messages limited to 140 characters. 

# (1) Using "twitter_samples" from nltk
 - NLTK's Twitter corpus currently contains a sample of 10k Tweets (named 'twitter_samples') which are divided according to sentiment into negative and positive
 - Tweets are stored in the form of .json
 - negative_tweets.json = 5k, positive_tweets.json = 5k, tweets.20150430-223406.json = 20k

In [1]:
import nltk                                     ## import Natural Language Tool Kit
#nltk.download("twitter_samples")               ## download and unzip all the text files of twitter samples
from nltk.corpus import twitter_samples as ts   ## import twitter_samples text files as ts
ts.abspath                                      ## location of files where they are stored

<bound method CorpusReader.abspath of <TwitterCorpusReader in 'C:\\Users\\Sai\\AppData\\Roaming\\nltk_data\\corpora\\twitter_samples'>>

Sample tweets from nltk twitter_samples corpus

In [2]:
import json                                            ## import json package

pos_tweets = ts.strings('positive_tweets.json')        ## 5k tweets with positive sentiment

neg_tweets = ts.strings('negative_tweets.json')        ## 5k tweets with negative sentiment
 
all_tweets = ts.strings('tweets.20150430-223406.json') ## 20k positive and negative tweets

print("POS = ", pos_tweets[0])           
print("POS = ", pos_tweets[1])          
print("NEG = ", neg_tweets[0])           
print("NEG = ", neg_tweets[1])           
print(all_tweets[0])              

POS =  #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
POS =  @Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!
NEG =  hopeless for tmr :(
NEG =  Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in 2 months :(
RT @KirkKus: Indirect cost of the UK being in the EU is estimated to be costing Britain £170 billion per year! #BetterOffOut #UKIP


<space>

<space>

### Creating a categorized text corpus :
For training, we need to first create a list of labeled feature sets.

We will convert the Categorized Plaintext Corpus Reader data into python list.

This list should be of the form [(featureset, label)], where the featureset variable is a dict and label is the known class label for the featureset.

In [3]:
df = ([(t, "pos") for t in pos_tweets] + 
             [(t, "neg") for t in neg_tweets])   ## categorize the text corpus, add labels to text
print(df[0])                                                             ## sample tweet with postitive sentiment
print(df[5000])                                                          ## sample tweet with negative sentiment

('#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)', 'pos')
('hopeless for tmr :(', 'neg')


In [4]:
len(df)

10000

### Splitting the data
In our dataset first 5000 tweets have positive sentiment and later have negative sentiment. So we will shuffle the data using shuffle

In [5]:
import random
random.seed(123)                          ## for reproducing same results each time


from random import shuffle                ## import required package/ functions
shuffle(df)                               ## shuffle the ordered reviews

Now we divide the dataset into train and test.

Train (75%) data will be processed in order to use it for training a classification model
Test (25%) data will be used for predicting the sentiment out of the review by the model built

In [6]:
ttrain=df[:7500]                           ## 75% of the total data
ttest=df[7500:]                            ## remaining 25% of the total data
print(len(ttrain),":",len(ttest))

7500 : 2500


# (2) Processing the data :

### Labels
**Extract the sentiment labels by making positive reviews as class 1 and negative reviews as class 2**<br>
This function will take the data in the form of [features, labels] and extracts labels as separate list with levels 1 and 2


### Extract the sentiment labels by making positive reviews as class 1 and negative reviews as class -1<br>
This function will take the data in the form of [features, labels] and extracts labels as separate list with levels 1 and -1

In [7]:
def get_lables(data):
    labels = []
    for tup in data:
        if tup[1].lower()=="neg":
            labels.append(-1)
        else:
            labels.append(1)
    return labels


These categories are exclusive, which makes a classifier trained on them a binary classifier.<br> 
Binary classifiers have only two classification labels, and will always choose one or the other.

In [8]:
ttraining_labels = get_lables(ttrain)

ttest_gold_labels = get_lables(ttest)

## Cleaning tweet data

Language used Twitter is used via a variety of media including SMS and mobile phone apps. Because of this and the 140-character limit, language used in Tweets tend be more colloquial, and filled with slang and misspellings. Use of hashtags also gained popularity on Twitter and is a primary feature in any given tweet.

**sample tweet:**
RT@soacecabard TOday's the Day. ICS <3
#andriod#google#samsung

**attributes:**<br>
**RT** = retweet symbol<br>
**@spacecanard** = handle<br>
**Today's the Dat.** = "." punctuation<br>
**<3** = emoticon<br>
**#andriod** = hashtag

<space>

- User-generated content on the web is seldom present in a form usable for learning. 
- It becomes important to normalize the text by applying a series of pre-processing steps. 
- Following pre-processing steps will be performed to decrease the size of the feature set to make it suitable for learning algorithms.

### HTML decoding
It looks like HTML encoding has not been converted to text, and ended up in text field as ‘&amp’,’&quot’,etc. Decoding HTML to general text will be my first step of data preparation. I will use BeautifulSoup for this.

In [9]:
from bs4 import BeautifulSoup
# for removing html coding in strings of the text
text = "Whinging. My client&amp;boss don't understand English well. Rewrote some text unreadable. It's written by v. good writer&amp;reviewed correctly. "
example1 = BeautifulSoup(text, 'lxml')
example1
print(example1.get_text())

Whinging. My client&boss don't understand English well. Rewrote some text unreadable. It's written by v. good writer&reviewed correctly. 


In [10]:
dfp = [(t) for t in ts.strings("positive_tweets.json")]
dfp[:5]

['#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)',
 '@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!',
 '@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!',
 '@97sides CONGRATS :)',
 'yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days']

### @ - handle
very Twitter user has a unique username. Any thing directed towards that user can be indicated be writing their username preceded by ‘@’. Thus, these are like proper nouns. For example, @Apple

Regular Expression: @(\w+)

In [11]:
import re
for i in range(5):
    print(re.sub(r'@[_A-Za-z0-9]+','', dfp[i]))

#FollowFriday    for being top engaged members in my community this week :)
 Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!
 we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!
 CONGRATS :)
yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days


### URL links
The third part of the cleaning is dealing with URL links, same with @mention, even though it carries some information, for sentiment analysis purpose, this can be ignored.

In [12]:
import re
for i in range(5):
    print(re.sub('https?://[A-Za-z0-9./]+','',dfp[i]))
# Regular Expression: (http|https|ftp)://[a-zA-Z0-9\\./]+

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!
@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!
@97sides CONGRATS :)
yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days


### Hashtag
A hashtag is a word or an un-spaced phrase prefixed with the hash symbol (#). These are used to both naming subjects and phrases that are currently in trending topics. For example, #iPad, #news

`Regular Expression: #(\w+)`

In [13]:
letters_only = re.sub("#(\w+)", " ", dfp[0])
letters_only

'  @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)'

### Emoticons
Use of emoticons is very prevalent throughout the web, more so on micro- blogging sites. We identify the following emoticons and replace them with a single word. Table 4 lists the emoticons we are currently detecting. All other emoticons would be ignored.

Happy emoticons are different versions of smiling face, like ":)", ":-)", ": )", ":D", "=)" etc. 

Sad emoticons include frowns, like ":(", ":-(", ":(" etc.

In [14]:
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)

### Repeating Characters
People often use repeating characters while using colloquial language, like "I’m in a hurrryyyyy", "We won, yaaayyyyy!" As our final pre-processing step, we replace characters repeating more than twice as two characters.

`Regular Expression: (.)\1{1,}`

### Punctuations
Although not all Punctuations are important from the point of view of classification but some of these, like question mark, exclamation mark can also provide information about the sentiments of the text. We replace every word boundary by a list of relevant punctuations present at that point. Table 5 lists the punctuations currently identified. We also remove any single quotes that might exist in the text.

`Punctuations	Examples
 PUNC_DOT	     .	
 PUNC_EXCL	    !	   ¡
 PUNC_QUES	    ?	   ¿
 PUNC_ELLP	    ...     …`

<space>

## Tokenize Tweets
NLTK has a TweetTokenizer module that does a good job in tokenizing (splitting text into a list of words) tweets.

Three different parameters can be passed while calling the TweetTokenizer class. They are:

- preserve_case: if False then it converts tweet to lowercase and vice-versa.
- strip_handles: if True then it removes twitter handles from the tweet and vice-versa.
- reduce_len: if True then it reduces the length of words in the tweet like hurrayyyy, yipppiieeee, etc. and vice-versa.

In [15]:
from nltk.tokenize import TweetTokenizer
tweet_tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
 
for tweet in pos_tweets[:5]:
    print (tweet_tokenizer.tokenize(tweet))

['#followfriday', 'for', 'being', 'top', 'engaged', 'members', 'in', 'my', 'community', 'this', 'week', ':)']
['hey', 'james', '!', 'how', 'odd', ':/', 'please', 'call', 'our', 'contact', 'centre', 'on', '02392441234', 'and', 'we', 'will', 'be', 'able', 'to', 'assist', 'you', ':)', 'many', 'thanks', '!']
['we', 'had', 'a', 'listen', 'last', 'night', ':)', 'as', 'you', 'bleed', 'is', 'an', 'amazing', 'track', '.', 'when', 'are', 'you', 'in', 'scotland', '?', '!']
['congrats', ':)']
['yeaaah', 'yipppy', '!', '!', '!', 'my', 'accnt', 'verified', 'rqst', 'has', 'succeed', 'got', 'a', 'blue', 'tick', 'mark', 'on', 'my', 'fb', 'profile', ':)', 'in', '15', 'days']


#### Cleaning
- Remove stock market tickers like $GE
- Remove retweet text “RT”
- Remove hyperlinks
- Remove hashtags (only the hashtag # and not the word)
- Remove stop words like a, and, the, is, are, etc.
- Remove emoticons like :), :D, :(, :-), etc.
- Remove punctuation like full-stop, comma, exclamation sign, etc.
- Convert words to Stem/Base words using Porter Stemming Algorithm. E.g. words like ‘working’, ‘works’, and ‘worked’ will be converted to their base/stem word “work”.

In [16]:
## importing required libraries
import string
import re
 
from nltk.corpus import stopwords 
stopwords_english = stopwords.words('english')
 
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
 
from nltk.tokenize import TweetTokenizer

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [94]:
def clean_tweets(tweet):
    
    tweet = re.sub(r'\$\w*', '', tweet)           ## remove stock market tickers like $GE £GE
    tweet = re.sub(r'^RT[\s]+', '', tweet)        ## remove old style retweet text "RT"
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)    ## remove hyperlinks
    tweet = re.sub(r'#', '', tweet)               ## remove hashtags sign from the word
    tweet = re.sub(r'\d+','', tweet)              ## remove numbers
    tweet = re.sub(r'[.][.][.]','', tweet)        ## remove special punctuations
    tweet = re.sub(r'[.][ ][.][ ][.]','', tweet)  ## remove special punctuations
    tweet = re.sub(r'(.\n.)','', tweet)           ## remove special punctuations
    tweet = re.sub(r'[“] | [”]','', tweet)        ## remove special punctuations
    tweet = re.sub(r'@[_A-Za-z0-9]+','', tweet)   ## remove handlers
    tweet = re.sub(r'_','', tweet)  ## remove special punctuations
    tweet = tweet.replace('\n', '')               ## replace new line html decoder
    
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
 
    tweets_clean = []    
    for word in tweet_tokens:
        if (word not in stopwords_english and       ## remove stopwords
              word not in emoticons and             ## remove emoticons
                word not in string.punctuation):    ## remove punctuation
            #tweets_clean.append(word)
            words_lemma = lemmatizer.lemmatize(word) ## lemmatizer
            #stem_word = stemmer.stem(word)          ## stemming word
            tweets_clean.append(words_lemma)
            
    return tweets_clean

In [95]:
custom_tweet = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"
 
# print cleaned tweet
print (clean_tweets(custom_tweet))

['hello', 'great', 'day', 'good', 'morning']


In [96]:
print (pos_tweets[5])

# print cleaned tweet
print (clean_tweets(pos_tweets[5]))

@BhaktisBanter @PallaviRuhail This one is irresistible :)
#FlipkartFashionFriday http://t.co/EbZ0L2VENM
['one', 'irresistible', 'lipkartfashionfriday']


In [97]:
print (ttrain[5])

# print cleaned tweet
print (clean_tweets(ttrain[5][0]))

('@GrahamTownsend More like an Ewok in my case :D @MiamiTrue', 'pos')
['like', 'ewok', 'case']


Lets unpack the train data and apply above pre-processing

In [98]:
df_pos_train_text = []
for a, b in ttrain:
    #print(a)
    df_pos_train_text.append(a)
    
len(df_pos_train_text)

7500

In [99]:
print (clean_tweets(df_pos_train_text[0]))

['hello', 'n', 'america', 'california', 'usa', 'dear', 'friend', 'thank', 'following']


In [100]:
clean_tweets_set = []
for tweet in range(len(df_pos_train_text)):
    clean_tweets_set.append((clean_tweets(df_pos_train_text[tweet])))

In [101]:
clean_tweets_set[:10]

[['hello',
  'n',
  'america',
  'california',
  'usa',
  'dear',
  'friend',
  'thank',
  'following'],
 ['stuff', 'well', 'would', 'know'],
 ['hate', 'waking', 'middle', 'sleep'],
 ['he',
  'adorable',
  'like',
  'cute',
  'marshmallow',
  'u',
  'wanna',
  'hug',
  'eat',
  'bc',
  'cute',
  'chswiyfxcskcalum',
  'love',
  'much'],
 ['jgh', 'bonding', 'niggs', 'bonakid'],
 ['like', 'ewok', 'case'],
 ['followfriday', 'top', 'support', 'community', 'week'],
 ['call', 'savage', 'god'],
 ['bath', 'even', '’', 'wheelie', 'bin', 'nowhere', 'put'],
 ['stats', 'day', 'arrived', 'new', 'follower', 'unfollowers', 'via']]

In [102]:
# This code will make above list of list into flat list
flat_list = [item for sublist in clean_tweets_set for item in sublist]
flat_list[:50]

['hello',
 'n',
 'america',
 'california',
 'usa',
 'dear',
 'friend',
 'thank',
 'following',
 'stuff',
 'well',
 'would',
 'know',
 'hate',
 'waking',
 'middle',
 'sleep',
 'he',
 'adorable',
 'like',
 'cute',
 'marshmallow',
 'u',
 'wanna',
 'hug',
 'eat',
 'bc',
 'cute',
 'chswiyfxcskcalum',
 'love',
 'much',
 'jgh',
 'bonding',
 'niggs',
 'bonakid',
 'like',
 'ewok',
 'case',
 'followfriday',
 'top',
 'support',
 'community',
 'week',
 'call',
 'savage',
 'god',
 'bath',
 'even',
 '’',
 'wheelie']

## Bag of words

Bag-of-words model that allows us to represent text as numerical feature vectors. 

The idea behind the bag-of-words model is quite simple and can be summarized as follows:
1. We create a vocabulary of unique tokens—for example, words—from the entire set of documents.
2. We construct a feature vector from each document that contains the counts of how often each word occurs in the particular document.

### Vocabulary
This function will create a vocabulary for each review and use it to get unigram features from the data

In [103]:
#Required for Bag of words (unigram features) creation
vocabulary = list(set(flat_list))
vocabulary.sort() #sorting the list
print(len(vocabulary))
print(vocabulary[0:500])

9301
['(-:', '(:', '):', ');', '--->', '-->', '->', '. .', '. ..', '..', ":'D", ':-\\', ':/', '::', ':op', ':|', ';-)', ';d', ';p', ';}', '<--', '<---', '=:', '>:-(', '>:D', 'D:', 'D;', 'a-foot', 'aa', 'aaa', 'aaaaaaaaah', 'aaaahhh', 'aaaand', 'aaah', 'aaahh', 'aaahhh', 'aahhh', 'aameell', 'aapke', 'aaron', 'aarwwi', 'aasramany', 'aayegi', 'ab', 'abandoned', 'abby', 'abby.can', 'abbymill', 'abes', 'abhi', 'abi', 'abit', 'able', 'abligaverins', 'abp', 'abroad', 'abrupt', 'abscess', 'absolute', 'absolutely', 'abstinence', 'abt', 'abu', 'abuti', 'ac', 'academic', 'acads', 'acc', 'accent', 'access', 'accessory', 'accident', 'accidentally', 'accnt', 'accomplishment', 'according', 'account', 'accurate', 'acdc', 'acha', 'achebe', 'achi', 'achieved', 'achieving', 'ack', 'acne', 'acorn', 'acoustic', 'across', 'act', 'action', 'activation', 'active', 'activity', 'actor', 'actress', 'actual', 'actually', 'actuallythough', 'acube', 'ad', 'ada', 'adam', 'add', 'added', 'addicted', 'addicted-to-anal

### Unigram Features
This function will get unigram features from the data.<br>
Prepare a unigram feature vector based on the presence or absence of words

In [27]:
def get_unigram_features(data, vocab):
    fet_vec_all = []
    for tup in data:
        single_feat_vec = []
        sent = tup[0].lower()                  ## Lowercasing the dataset
        for v in vocab:
            if sent.__contains__(v):
                single_feat_vec.append(1)
            else:
                single_feat_vec.append(0)
        fet_vec_all.append(single_feat_vec)
    return fet_vec_all

In [28]:
## code for printing time in seconds for execution of each cell
import time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))  ## time consumed = start time - end time

--- 0.0 seconds ---


In [29]:
start_time = time.time()

training_unigram_features = get_unigram_features(ttrain, vocabulary)                      # vocabulary extracted in the beginning
test_unigram_features = get_unigram_features(ttest, vocabulary)

print("--- %s seconds ---" % (time.time() - start_time))

--- 31.500887632369995 seconds ---


### Sentiword net
Lets add more features from sentiwordnet

In [30]:
from nltk.corpus import sentiwordnet as swn
#nltk.download("all")
swn.abspath

<bound method CorpusReader.abspath of <SentiWordNetCorpusReader in 'C:\\Users\\Sai\\AppData\\Roaming\\nltk_data\\corpora\\sentiwordnet'>>

In [31]:
# from nltk.corpus import sentiwordnet as swn
# nltk.download("sentiwordnet")

def get_senti_wordnet_features(data):
    fet_vec_all = []
    for tup in data:
        sent = tup[0].lower()
        words = sent.split()
        pos_score = 0
        neg_score = 0
        for w in words:
            senti_synsets = swn.senti_synsets(w.lower())
            for senti_synset in senti_synsets:
                p = senti_synset.pos_score()
                n = senti_synset.neg_score()
                pos_score+=p
                neg_score+=n
                break #take only the first synset (Most frequent sense)
        fet_vec_all.append([float(pos_score),float(neg_score)])
    return fet_vec_all

In [32]:
start_time = time.time()

training_swn_features = get_senti_wordnet_features(ttrain)
test_swn_features     = get_senti_wordnet_features(ttest)

print("--- %s seconds ---" % (time.time() - start_time))

--- 11.14002776145935 seconds ---


**Merge the features**

In [33]:
def merge_features(featureList1,featureList2):
    # For merging two features
    if featureList1==[]:
        return featureList2
    merged = []
    for i in range(len(featureList1)):
        m = featureList1[i]+featureList2[i]
        merged.append(m)
    return merged

In [34]:
start_time = time.time()

ttraining_features = merge_features( training_unigram_features, training_swn_features)
ttest_features     = merge_features( test_unigram_features,     test_swn_features)

## These are the final set of features on which training and testing can ber performed
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.196302890777588 seconds ---


<space>

<space>

# (3) Training Model

This function will calculate the prediction(classification) precision with reference to actual labels

In [35]:
def calculate_precision(prediction, actual):
    prediction = list(prediction)
    correct_labels = [prediction[i]  for i in range(len(prediction)) if actual[i] == prediction[i]]
    precision = float(len(correct_labels))/float(len(prediction))
    return precision

### Naive Bayes Model

In [36]:
start_time = time.time()

from sklearn.naive_bayes import MultinomialNB
nb_classifier = MultinomialNB(alpha=0.1).fit(ttraining_features, ttraining_labels) #training process

print("Precision of NB classifier is")

pred = nb_classifier.predict(ttraining_features)
precision = calculate_precision(pred, ttraining_labels)
print("Training data\t" + str(precision))

pred = nb_classifier.predict(ttest_features)
twitter_nb = calculate_precision(pred, ttest_gold_labels)
print("Test data\t" + str(twitter_nb))

print("--- %s seconds ---" % (time.time() - start_time))

Precision of NB classifier is
Training data	0.8834666666666666
Test data	0.7376
--- 14.028931140899658 seconds ---


### Decision Tree Classifier

In [37]:
start_time = time.time()

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=5, max_features='sqrt', random_state=42).fit(ttraining_features, ttraining_labels)

print("Precision of Decision Tree classifier is")

pred = clf.predict(ttraining_features)
precision = calculate_precision(pred, ttraining_labels)
print("Training data\t" + str(precision))

pred = clf.predict(ttest_features)
twitter_dt = calculate_precision(pred, ttest_gold_labels)
print("Test data\t" + str(twitter_dt))

print("--- %s seconds ---" % (time.time() - start_time))

Precision of Decision Tree classifier is
Training data	0.5933333333333334
Test data	0.5852
--- 13.119225025177002 seconds ---


### Logistic Regression

In [38]:
start_time = time.time()

from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=42).fit(ttraining_features, ttraining_labels)

print("Precision of Logistic Regression is")

pred = clf.predict(ttraining_features)
precision = calculate_precision(pred, ttraining_labels)
print("Training data\t" + str(precision))

pred = clf.predict(ttest_features)
twitter_lr = calculate_precision(pred, ttest_gold_labels)
print("Test data\t" + str(twitter_lr))
print("--- %s seconds ---" % (time.time() - start_time))

Precision of Logistic Regression is
Training data	0.9290666666666667
Test data	0.758
--- 13.952841758728027 seconds ---


### SVM Model

In [70]:
start_time = time.time()

from sklearn.svm import SVC
clf = SVC(C=0.01, kernel='sigmoid').fit(ttraining_features, ttraining_labels)

print("Precision of SVM classifier is")

pred = clf.predict(ttraining_features)
precision = calculate_precision(pred, ttraining_labels)
print("Training data\t" + str(precision))

pred = clf.predict(mtest_features)
twitter_svm = calculate_precision(pred, ttest_gold_labels)
print("Test data\t" + str(twitter_svm))
print("--- %s seconds ---" % (time.time() - start_time))

Precision of SVM classifier is
Training data	0.5022857142857143
Test data	0.49466666666666664
--- 2708.3680284023285 seconds ---


# (1) Sentiment Analysis on Movie Review Corpus

**Task 1:** The goal here is to predict the sentiments behind movie reviews

# (1) Using "movie_reviews" from nltk
### movie_reviews corpus : 
- 2000 movie reviews which are stored in separate text files with sentiment polarity classification tag.
   <br>for eg. neg/cv000_29416.txt', 'pos/cv000_29590.txt'
- This corpus contains two categories of text: pos (a positive sentiment) and neg (a negative sentiment). 
- First 1000 reviews are negative and later are positives.

In [41]:
import nltk                                     ## import Natural Language Tool Kit
#nltk.download("all")                           ## download all the packages and text files from nltk
#nltk.download("movie_reviews")                 ## download and unzip all the text files of movie_reviews
from nltk.corpus import movie_reviews as mr     ## import movie_review text files as mr
mr.abspath                                      ## location of files where they are stored

<bound method CorpusReader.abspath of <CategorizedPlaintextCorpusReader in 'C:\\Users\\Sai\\AppData\\Roaming\\nltk_data\\corpora\\movie_reviews'>>

In [42]:
# mr.fileids("pos")                             ## list of files with positive sentiment
# mr.fileids("neg")                             ## list of files with negative sentiment

## Creating a categorized text corpus :

For training, we need to first create a list of labeled feature sets.<br>

We will convert the Categorized Plaintext Corpus Reader data into python list.<br>

This list should be of the form [(featureset, label)], where the featureset variable is a dict and label is the known class label for the featureset.<br> 


In [43]:
reviews = []                              ## create an empty list

for fileid in mr.fileids():
    tag, filename = fileid.split('/')     ## extracting featureset(text) and its corresponding labels from file name
    reviews.append([mr.raw(fileid), tag]) ## appending the featureset to the list in the form of [(featureset, label)]
    
len(reviews)                              ## length of list created; 0-999 = neg, 1000-1999 = pos

2000

In [44]:
#print(reviews[999])                       ## sample of negative sentiment review list
#print(reviews[1000])                      ## sample of positive sentiment review list

<space>

## Splitting the data

In our dataset first 1000 reviews are negative and later are positives. So we will shuffle the data using shuffle

In [45]:
import random
random.seed(123)                          ## for reproducing same results each time


from random import shuffle                ## import required package/ functions
shuffle(reviews)                          ## shuffle the ordered reviews

Now we divide the dataset into train and test.<br>
- Train (75%) data will be processed in order to use it for training a classification model<br>
- Test (25%) data will be used for predicting the sentiment out of the review by the model built

In [46]:
mtrain = reviews[:1500]                    ## 75% of the total data
mtest  = reviews[1500:]                    ## remaining 25% of the total data
print(len(mtrain),":",len(mtest))

1500 : 500


<space>

# (2) Processing the data :
- We have to convert categorical data, such as text or words, into a numerical form before we can pass it on to a machine learning algorithm.<br>
- We will use training data as reference for processing

In [47]:
mtraining_labels = get_lables(mtrain)

mtest_gold_labels = get_lables(mtest)

In [48]:
##len(mtraining_labels)

## Lets build bag of words from training data

In [49]:
## This will remove the tags from training data, only text will be stores in df
df = []
for a, b in mtrain:
    #print(a)
    df.append(a)
    
len(df)

1500

### Html decoding

In [50]:
df[0]

'most movies seem to release a third movie just so it can be called a trilogy . \nrocky iii seems to kind of fit in that category , but manages to be slightly unique . \nthe rocky formula of " rocky loses fight/rocky trains/rocky wins fight " is carried out to the letter . \nalso the \'tradition\' of showing the last five minutes or so from the past rocky film is used as well . \nthis movie begins with a series of clips showing how famous rocky ( sylvester stallone ) has become . . . \neven showing a brief appearance on sesame street ! ! \nthen it moves on with rocky being in a fixed fight with thunderlips ( hulk hogan ) . \na mysterious bad-ass known as clubber lang ( mr . t ) trash-talks to rocky about his stupid decision to retire from boxing and that he " pities the fool " for not coming out and fighting him outright . \nrocky\'s trainer ( burgess meredith ) tells rocky not to fight , but the italian stallion doesn\'t listen . \nnaturally , he gets his ass kicked . \nsomewhere alon

In [51]:
from bs4 import BeautifulSoup
# for removing html coding in strings of the text
example1 = BeautifulSoup(df[0], 'lxml')
example1
print(example1.get_text())

most movies seem to release a third movie just so it can be called a trilogy . 
rocky iii seems to kind of fit in that category , but manages to be slightly unique . 
the rocky formula of " rocky loses fight/rocky trains/rocky wins fight " is carried out to the letter . 
also the 'tradition' of showing the last five minutes or so from the past rocky film is used as well . 
this movie begins with a series of clips showing how famous rocky ( sylvester stallone ) has become . . . 
even showing a brief appearance on sesame street ! ! 
then it moves on with rocky being in a fixed fight with thunderlips ( hulk hogan ) . 
a mysterious bad-ass known as clubber lang ( mr . t ) trash-talks to rocky about his stupid decision to retire from boxing and that he " pities the fool " for not coming out and fighting him outright . 
rocky's trainer ( burgess meredith ) tells rocky not to fight , but the italian stallion doesn't listen . 
naturally , he gets his ass kicked . 
somewhere along the line afte

### Cleaning
- Remove stock market tickers like $GE
- Remove hyperlinks
- Remove hashtags (only the hashtag # and not the word)
- Remove stop words like a, and, the, is, are, etc.
- Remove emoticons like :), :D, :(, :-), etc.
- Remove punctuation like full-stop, comma, exclamation sign, etc.
- Convert words to Stem/Base words using nltk lemmatizer. E.g. words like ‘working’, ‘works’, and ‘worked’ will be converted to their base/stem word “work”.

In [104]:
clean_words_set = []
for word in range(len(df)):
    clean_words_set.append((clean_tweets(df[word])))

In [105]:
clean_words(df[0])

['movie',
 'seem',
 'release',
 'third',
 'movie',
 'called',
 'trilogy',
 'rocky',
 'iii',
 'seems',
 'kind',
 'fit',
 'category',
 'manages',
 'slightly',
 'unique',
 'rocky',
 'formula',
 'rocky',
 'loses',
 'fight',
 'rocky',
 'train',
 'rocky',
 'win',
 'fight',
 'carried',
 'letter',
 'also',
 'tradition',
 'showing',
 'last',
 'five',
 'minute',
 'past',
 'rocky',
 'film',
 'used',
 'well',
 'movie',
 'begin',
 'series',
 'clip',
 'showing',
 'famous',
 'rocky',
 'sylvester',
 'stallone',
 'become',
 '. . .',
 'even',
 'showing',
 'brief',
 'appearance',
 'sesame',
 'street',
 'move',
 'rocky',
 'fixed',
 'fight',
 'thunderlips',
 'hulk',
 'hogan',
 'mysterious',
 'bad-ass',
 'known',
 'clubber',
 'lang',
 'mr',
 'trash-talks',
 'rocky',
 'stupid',
 'decision',
 'retire',
 'boxing',
 'pity',
 'fool',
 'coming',
 'fighting',
 'outright',
 "rocky's",
 'trainer',
 'burgess',
 'meredith',
 'tell',
 'rocky',
 'fight',
 'italian',
 'stallion',
 'listen',
 'naturally',
 'get',
 'as',
 

In [106]:
clean_words_set[0]

['movie',
 'seem',
 'release',
 'third',
 'movie',
 'called',
 'trilogy',
 'ocky',
 'iii',
 'seems',
 'kind',
 'fit',
 'category',
 'manages',
 'slightly',
 'unique',
 'rocky',
 'formula',
 'rocky',
 'loses',
 'fight',
 'rocky',
 'train',
 'rocky',
 'win',
 'fight',
 'carried',
 'letter',
 'lso',
 'tradition',
 'showing',
 'last',
 'five',
 'minute',
 'past',
 'rocky',
 'film',
 'used',
 'well',
 'movie',
 'begin',
 'series',
 'clip',
 'showing',
 'famous',
 'rocky',
 'sylvester',
 'stallone',
 'become',
 'ven',
 'showing',
 'brief',
 'appearance',
 'sesame',
 'street',
 'hen',
 'move',
 'rocky',
 'fixed',
 'fight',
 'thunderlips',
 'hulk',
 'hogan',
 'mysterious',
 'bad-ass',
 'known',
 'clubber',
 'lang',
 'mr',
 'trash-talks',
 'rocky',
 'stupid',
 'decision',
 'retire',
 'boxing',
 'pity',
 'fool',
 'coming',
 'fighting',
 'outright',
 "ocky's",
 'trainer',
 'burgess',
 'meredith',
 'tell',
 'rocky',
 'fight',
 'italian',
 'stallion',
 'listen',
 'aturally',
 'get',
 'as',
 'kicked

In [107]:
# This code will make above list of list into flat list
flat_list = [item for sublist in clean_words_set for item in sublist]
flat_list[:50]

['movie',
 'seem',
 'release',
 'third',
 'movie',
 'called',
 'trilogy',
 'ocky',
 'iii',
 'seems',
 'kind',
 'fit',
 'category',
 'manages',
 'slightly',
 'unique',
 'rocky',
 'formula',
 'rocky',
 'loses',
 'fight',
 'rocky',
 'train',
 'rocky',
 'win',
 'fight',
 'carried',
 'letter',
 'lso',
 'tradition',
 'showing',
 'last',
 'five',
 'minute',
 'past',
 'rocky',
 'film',
 'used',
 'well',
 'movie',
 'begin',
 'series',
 'clip',
 'showing',
 'famous',
 'rocky',
 'sylvester',
 'stallone',
 'become',
 'ven']

## Bag of words

Bag-of-words model that allows us to represent text as numerical feature vectors. 

The idea behind the bag-of-words model is quite simple and can be summarized as follows:
1. We create a vocabulary of unique tokens—for example, words—from the entire set of documents.
2. We construct a feature vector from each document that contains the counts of how often each word occurs in the particular document.

### Vocabulary
This function will create a vocabulary for each review and use it to get unigram features from the data

In [108]:
#Required for Bag of words (unigram features) creation
vocabulary = list(set(flat_list))
vocabulary.sort() #sorting the list
print(len(vocabulary))
print(vocabulary[0:500])

41247
['\x05', '\x12', '\x13', '\x14', '\x16', '.  .', '. .', '..', '...', "a's", 'a-bomb', 'a-changin', 'a-flailing', "a-ha's", 'a-hole', 'a-list', 'a-plenty', 'a-trois', 'a-week', 'aa-meetings', 'aaa', 'aaah', 'aaahhhs', 'aahs', 'aaliyah', "aaliyah's", 'aalyah', "aalyah's", 'aamir', 'aan', 'aardman', 'aarky', 'aaron', 'aast', 'aatish', "aaya's", 'ab', 'aback', 'abandon', 'abandoned', 'abandoning', 'abandonment', 'abating', 'abba', 'abbe', 'abberation', 'abberline', 'abbot', 'abbott', 'abbotts', 'abbreviated', 'abby', "abby's", "abc's", 'abdomen', 'abducted', 'abduction', 'abdul-jabbar', 'abe', 'abel', 'abeled', 'aberdeen', 'aberration', 'abetting', 'abeyance', 'abhorrence', 'abhorrent', 'abider', 'abides', 'abigail', "abigail's", 'abiility', 'ability', 'abject', 'ablaze', 'able', 'ably', 'abney', 'abnormally', 'aboard', 'abode', 'abolish', 'abolitionist', 'abominable', 'abomination', 'aboo', 'aborginal', 'aboriginal', 'aboring', 'abort', 'aborted', 'abortion', 'abortionist', 'abortiv

### Unigram Features

In [61]:
start_time = time.time()

training_unigram_features = get_unigram_features(mtrain,vocabulary) # vocabulary extracted in the beginning
test_unigram_features     = get_unigram_features(mtest,vocabulary)

print("--- %s seconds ---" % (time.time() - start_time))

--- 314.4381802082062 seconds ---


### Sentiwordnet


In [18]:
start_time = time.time()

training_swn_features = get_senti_wordnet_features(mtrain)
test_swn_features     = get_senti_wordnet_features(mtest)

print("--- %s seconds ---" % (time.time() - start_time))

--- 87.85296511650085 seconds ---


### Merging the features

In [63]:
start_time = time.time()

mtraining_features = merge_features( training_unigram_features, training_swn_features)
mtest_features     = merge_features( test_unigram_features,     test_swn_features)

## These are the final set of features on which training and testing can ber performed
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.0876069068908691 seconds ---


# (3) Training Model

### Naive Bayes Model

In [64]:
start_time = time.time()

from sklearn.naive_bayes import MultinomialNB
nb_classifier = MultinomialNB(alpha=0.1).fit(mtraining_features, mtraining_labels) #training process

print("Precision of NB classifier is")

pred = nb_classifier.predict(mtraining_features)
precision = calculate_precision(pred, mtraining_labels)

print("Training data\t" + str(precision))
pred = nb_classifier.predict(mtest_features)

movie_nb = calculate_precision(pred, mtest_gold_labels)
print("Test data\t" + str(movie_nb))
print("--- %s seconds ---" % (time.time() - start_time))

Precision of NB classifier is
Training data	0.99
Test data	0.836
--- 19.87365460395813 seconds ---


### Decision Tree Classifier

In [65]:
start_time = time.time()

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=5, max_features='sqrt', random_state=42).fit(mtraining_features, mtraining_labels)

print("Precision of Decision Tree classifier is")

pred = clf.predict(mtraining_features)
precision = calculate_precision(pred, mtraining_labels)
print("Training data\t" + str(precision))

pred = clf.predict(mtest_features)
movie_dt = calculate_precision(pred, mtest_gold_labels)
print("Test data\t" + str(movie_dt))
print("--- %s seconds ---" % (time.time() - start_time))

Precision of Decision Tree classifier is
Training data	0.6226666666666667
Test data	0.588
--- 13.158593654632568 seconds ---


### Logistic Regression

In [66]:
start_time = time.time()
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=42).fit(mtraining_features, mtraining_labels)
print("Precision of Logistic Regression is")

pred = clf.predict(mtraining_features)
precision = calculate_precision(pred, mtraining_labels)
print("Training data\t" + str(precision))

pred = clf.predict(mtest_features)
movie_lr = calculate_precision(pred, mtest_gold_labels)
print("Test data\t" + str(movie_lr))
print("--- %s seconds ---" % (time.time() - start_time))

Precision of Logistic Regression is
Training data	1.0
Test data	0.83
--- 15.364763736724854 seconds ---


### SVM Model

In [67]:
start_time = time.time()
from sklearn.svm import SVC
clf = SVC(C=0.01, kernel='sigmoid').fit(mtraining_features, mtraining_labels)
print("Precision of SVM classifier is")

pred = clf.predict(mtraining_features)
precision = calculate_precision(pred, mtraining_labels)
print("Training data\t" + str(precision))

pred = clf.predict(mtest_features)
movie_svm = calculate_precision(pred, mtest_gold_labels)
print("Test data\t" + str(movie_svm))
print("--- %s seconds ---" % (time.time() - start_time))

Precision of SVM classifier is
Training data	0.5026666666666667
Test data	0.492
--- 347.9355294704437 seconds ---


## Scorecard

In [71]:
# Lets make score card for each model in tabular form using pandas
import pandas as pd

list_of_lists = []
list_of_lists.append(['Movie Review', movie_nb, movie_svm, movie_dt, movie_lr])
list_of_lists.append(['Twitter Dataset', twitter_nb, twitter_svm, twitter_dt, twitter_lr])


Score = pd.DataFrame(list_of_lists, columns=['Score','Naive Bayes','SVM', 'Decision Tree', 'Logistic Regression'])

Score = Score.set_index('Score')
Score

,Naive Bayes,SVM,Decision Tree,Logistic Regression
Score,,,,
Movie Review,0.8360,0.492000,0.5880,0.830
Twitter Dataset,0.7376,0.494667,0.5852,0.758
